# Rebuild data
Rebuilding data consists on filling invalid values (`NaN` or negatives, in our case) for each building's data. This is done applying two operations:
- **Substitution**. Any day when missing data is more than 1/3 of said day (8 consumptions) will be replaced by the corresponding prototype.
- **Transformation**. Otherwise, a transformation of the corresponding prototype will replace the invalid values.

#### Directory structure
./<br></br>
notebook/<br></br>
    &emsp;|--- data-preprocessing<br></br>
    &emsp;&emsp;&emsp;&emsp;|--- rebuild_data.ipynb<br></br>
out/<br></br>
    &emsp;|--- consumptions_byday/<br></br>
    &emsp;|--- consumptions/<br></br>

In [1]:
OUT_PATH = 'C:/Users/thmas/OneDrive - Universidad de Castilla-La Mancha/Informática/TFG/out/'
PROTOS_PATH = OUT_PATH + 'prototypesMEAN.zip'

In [2]:
import pandas as pd

In [3]:
protos = pd.read_pickle(PROTOS_PATH)
protos

,building_id,weekday,active,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,89,6,False,19.924768,20.044887,20.077262,20.159969,20.254587,20.194550,20.284663,...,20.539652,20.204600,19.908497,19.817461,19.490365,19.496418,19.411085,19.433544,19.419972,19.426788
1,89,0,True,29.024269,33.833840,36.353319,44.146153,47.689857,48.602982,47.334915,...,40.259998,37.713417,34.697437,32.557381,31.539096,30.958461,30.819581,30.918707,30.827001,30.487459
2,89,0,False,18.809065,20.889957,21.949280,26.206468,29.695707,30.004400,28.989793,...,23.136092,21.743418,21.135650,20.583311,20.127893,20.069345,20.112862,20.128729,20.053196,20.113060
3,89,1,True,31.603164,36.111243,38.318624,46.232699,50.135373,49.145355,47.396200,...,39.939749,37.473955,34.796319,32.533838,31.695102,31.112563,31.040633,30.835235,31.097579,30.481038
4,89,1,False,20.744649,22.705368,25.013126,28.601601,31.301887,31.550106,30.883649,...,23.208710,21.899220,21.170070,20.571534,20.158271,20.096993,20.083515,20.014828,19.915396,19.945977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2233,3,False,11.467596,11.422271,18.115797,24.280768,26.163997,26.056783,25.313085,...,12.098499,11.605185,10.951535,10.719605,10.721716,10.281587,10.922931,11.494247,11.664281,11.607698
1257,2233,4,True,20.295182,21.703070,26.496233,32.032129,34.694525,36.477349,35.954045,...,19.218090,18.586900,16.122553,15.228944,15.336057,14.993393,16.391621,16.178704,16.319751,16.181950
1258,2233,4,False,11.198024,11.084300,17.362477,23.497256,25.927395,26.779238,25.941232,...,11.933169,11.284585,10.406484,10.411684,10.558328,9.967281,10.291822,10.991506,11.389434,11.325878
1259,2233,5,True,21.780315,21.510202,21.377805,21.358941,21.385059,21.487320,21.435786,...,21.548785,21.626989,21.527360,21.378919,21.292476,21.355302,21.718329,21.676526,21.605639,21.692100


In [4]:
counter_id = 89 # Building ID example

mean_proto = protos[protos['building_id'] == counter_id]
mean_proto.reset_index(drop=True, inplace=True)
mean_proto

,building_id,weekday,active,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,89,6,False,19.924768,20.044887,20.077262,20.159969,20.254587,20.194550,20.284663,...,20.539652,20.204600,19.908497,19.817461,19.490365,19.496418,19.411085,19.433544,19.419972,19.426788
1,89,0,True,29.024269,33.833840,36.353319,44.146153,47.689857,48.602982,47.334915,...,40.259998,37.713417,34.697437,32.557381,31.539096,30.958461,30.819581,30.918707,30.827001,30.487459
2,89,0,False,18.809065,20.889957,21.949280,26.206468,29.695707,30.004400,28.989793,...,23.136092,21.743418,21.135650,20.583311,20.127893,20.069345,20.112862,20.128729,20.053196,20.113060
3,89,1,True,31.603164,36.111243,38.318624,46.232699,50.135373,49.145355,47.396200,...,39.939749,37.473955,34.796319,32.533838,31.695102,31.112563,31.040633,30.835235,31.097579,30.481038
4,89,1,False,20.744649,22.705368,25.013126,28.601601,31.301887,31.550106,30.883649,...,23.208710,21.899220,21.170070,20.571534,20.158271,20.096993,20.083515,20.014828,19.915396,19.945977
5,89,2,True,31.918586,36.885773,39.425207,47.593670,50.037948,48.837055,47.404120,...,40.078078,37.087400,34.739128,32.578833,31.927701,31.452351,31.058499,31.032793,31.152242,30.727937
6,89,2,False,21.564084,23.087109,24.705329,28.034704,30.447546,31.389647,30.811437,...,23.913081,22.437524,21.678715,21.039603,20.527422,20.565198,20.483317,20.445373,20.311296,20.345218
7,89,3,True,32.253841,36.732195,38.802914,46.401775,50.196061,49.436505,47.524532,...,39.858279,37.140220,34.718181,32.457888,31.641315,31.208599,30.820361,30.941222,30.804630,30.494814
8,89,3,False,21.276617,23.188231,25.183666,28.924441,31.283082,31.365324,30.925434,...,804.680917,21.629306,21.080358,20.509809,20.007613,19.947361,19.965557,19.841944,19.833062,19.888298
9,89,4,True,33.084816,37.722791,39.619311,46.575444,48.632746,48.472832,47.539280,...,39.977219,37.534106,34.705409,33.102889,32.406356,31.866627,31.853466,31.610200,31.679531,30.957044


In [5]:
raw_df = pd.read_pickle(OUT_PATH + 'raw_consumptions.zip')
raw_df

,building_id,weekday,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2012-02-24,89,4,NaN,NaN,NaN,NaN,0.000000,25.968207,34.830257,29.201536,...,15.390713,11.000000,13.000000,10.000000,8.000000,8.000000,8.000000,8.000000,8.000000,9.000000
2012-02-25,89,5,8.000000,8.569660,7.830417,7.830404,7.769520,8.895253,7.104747,8.000000,...,9.000000,8.268304,8.837209,9.894487,9.000000,9.617350,8.837233,9.545417,9.000000,8.000000
2012-02-26,89,6,9.000000,9.000000,8.478725,8.521275,9.000000,8.000000,9.000000,8.000000,...,10.000000,9.000000,8.203082,8.837309,8.959610,9.000000,8.552664,9.447336,9.000000,8.064059
2012-02-27,89,0,9.935941,9.000000,10.000000,18.413394,36.837317,35.749289,27.000000,25.762449,...,23.837638,16.837659,16.837655,14.837669,15.863704,14.000000,15.000000,15.000000,15.000000,14.000000
2012-02-28,89,1,15.000000,15.000000,15.000000,23.000000,41.347489,39.652511,34.021674,33.837309,...,24.000000,15.000000,15.000000,14.000000,14.068779,15.931221,15.000000,14.581427,14.418573,15.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-04,2233,2,22.909913,32.403865,33.628554,44.708887,41.996861,41.830621,39.931565,38.410110,...,16.404210,16.274590,10.265586,8.801078,8.523268,8.920408,8.643050,8.642317,8.806212,15.055494
2020-03-05,2233,3,19.531186,37.388927,40.617218,42.142622,44.729416,42.029408,41.356996,37.418092,...,14.259403,15.147316,9.511927,8.137753,8.261290,8.352191,8.169629,14.327638,12.734737,19.528338
2020-03-06,2233,4,24.619904,39.854596,38.970224,46.408685,50.081861,39.844651,40.825715,33.403195,...,22.267127,17.071403,11.609071,8.515376,8.416603,8.297641,8.482840,8.284570,8.371784,8.781833


In [6]:
df = raw_df[raw_df['building_id'] == counter_id]
df

,building_id,weekday,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2012-02-24,89,4,NaN,NaN,NaN,NaN,0.000000,25.968207,34.830257,29.201536,...,15.390713,11.000000,13.000000,10.000000,8.000000,8.000000,8.000000,8.000000,8.000000,9.000000
2012-02-25,89,5,8.000000,8.569660,7.830417,7.830404,7.769520,8.895253,7.104747,8.000000,...,9.000000,8.268304,8.837209,9.894487,9.000000,9.617350,8.837233,9.545417,9.000000,8.000000
2012-02-26,89,6,9.000000,9.000000,8.478725,8.521275,9.000000,8.000000,9.000000,8.000000,...,10.000000,9.000000,8.203082,8.837309,8.959610,9.000000,8.552664,9.447336,9.000000,8.064059
2012-02-27,89,0,9.935941,9.000000,10.000000,18.413394,36.837317,35.749289,27.000000,25.762449,...,23.837638,16.837659,16.837655,14.837669,15.863704,14.000000,15.000000,15.000000,15.000000,14.000000
2012-02-28,89,1,15.000000,15.000000,15.000000,23.000000,41.347489,39.652511,34.021674,33.837309,...,24.000000,15.000000,15.000000,14.000000,14.068779,15.931221,15.000000,14.581427,14.418573,15.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-09,89,5,25.941659,27.335455,25.121847,25.288444,24.678067,26.295092,26.057136,25.672894,...,24.674138,24.378837,24.009223,24.627292,24.965822,26.335574,25.091018,25.300881,24.706466,25.646667
2019-03-10,89,6,24.673825,24.706081,25.643515,24.669514,24.701949,24.644706,23.684258,24.679232,...,25.042024,24.005963,24.994202,23.674068,25.671791,25.672914,24.677309,25.674312,24.675508,26.666800
2019-03-11,89,0,23.666993,27.679830,32.340251,37.344219,41.345961,42.011335,45.333564,43.677254,...,33.011743,31.671082,27.338533,28.339967,23.998888,24.664691,23.688295,24.324001,24.002153,23.695071


In [7]:
def get_threshold(df: pd.DataFrame) -> float:
    df.loc[:, 'mean'] = df.loc[:, '0':].mean(axis=1) # Calculate daily consumption mean
    mean, std = df['mean'].mean(), df['mean'].std()
    
    del df['mean']
    
    return mean + 2 * std # Calculate threshold based on sundays

Obtain the threshold for the current building

In [8]:
sundays = df[df['weekday'] == 6]
threshold = get_threshold(sundays)
threshold

D:\PythonEnvironments\pytfg\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\PythonEnvironments\pytfg\lib\site-packages\pandas\core\indexing.py:964: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


34.7152279063718

### Substitution and Transformation
In order to get a "clean" dataset, that is, a dataset without any kind of invalid value (`Nan`, negatives...), a `transform(X, Y)` operation is applied to the whole dataset (day by day). If a whole day is invalid, it is simply replaced by the corresponding prototype (based on activity and weekday). Otherwise, the transformation is applied. This function consists on three steps:

- **Normalization**. $X$ and $Y$ are normalized.
- **Substitution**. Any invalid value on $Y$ is replaced with its corresponding $X$ value (same hour consumption).
- **Denormalization**. $Y$ is denormalized, to obtain the real consumptions.

In [9]:
def normalize(x: pd.DataFrame, mean: float, std: float) -> pd.DataFrame:
    return (x - mean) / std

In [10]:
def denormalize(x: pd.DataFrame, mean: float, std: float) -> pd.DataFrame:
    return x * std + mean

In [11]:
def transform(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
    nan = y.isna()
    negatives = y.lt(0)
    
    if not False in nan or not False in negatives:
        return x
    
    x_clean = x.dropna()
    x_mean, x_std = x_clean.mean(), x_clean.std()
    
    x_norm = normalize(x, x_mean, x_std)
    
    y_clean = y.dropna()
    y_mean, y_std = y_clean.mean(), y_clean.std()
    
    y_norm = normalize(y, y_mean, y_std)
    
    y_norm[nan] = x_norm[nan]
    y_norm[negatives] = x_norm[negatives]
           
    return denormalize(y_norm, y_mean, y_std)

In [12]:
for i in range(len(df)):
    weekday = df['weekday'].iloc[i]
    
    day_of_week = weekday == 6 or weekday == 5 # It is a Sunday or a Saturday
    august = df.index[i].month == 8 # August is an inactive month
    christmas = (df.index[i].month == 12 and df.index[i].day >= 23) or (df.index[i].month == 1 and df.index[i].day <= 6) # Christmas holidays
    
    inactive = day_of_week or august or christmas
    
    if inactive:
        df.iloc[i, 2:] = transform(mean_proto[(mean_proto['weekday'] == weekday) & (mean_proto['active'] == False)].iloc[0, 3:], df.iloc[i, 2:])
    else:
        df.iloc[i, 2:] = transform(mean_proto[(mean_proto['weekday'] == weekday) & (mean_proto['active'] == True)].iloc[0, 3:], df.iloc[i, 2:])
        
df

,building_id,weekday,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2012-02-24,89,4,33.084816,37.722791,39.619311,46.575444,48.632746,48.472832,47.539280,48.113746,...,39.977219,37.534106,34.705409,33.102889,32.406356,31.866627,31.853466,31.610200,31.679531,30.957044
2012-02-25,89,5,19.842774,19.942503,20.041401,20.849483,21.292216,21.110874,21.073111,21.071848,...,20.281598,19.920581,19.616548,19.447081,19.167541,19.108408,19.032248,19.116976,18.924928,18.920963
2012-02-26,89,6,19.924768,20.044887,20.077262,20.159969,20.254587,20.194550,20.284663,20.413814,...,20.539652,20.204600,19.908497,19.817461,19.490365,19.496418,19.411085,19.433544,19.419972,19.426788
2012-02-27,89,0,29.024269,33.833840,36.353319,44.146153,47.689857,48.602982,47.334915,46.795633,...,40.259998,37.713417,34.697437,32.557381,31.539096,30.958461,30.819581,30.918707,30.827001,30.487459
2012-02-28,89,1,31.603164,36.111243,38.318624,46.232699,50.135373,49.145355,47.396200,47.320543,...,39.939749,37.473955,34.796319,32.533838,31.695102,31.112563,31.040633,30.835235,31.097579,30.481038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-09,89,5,19.842774,19.942503,20.041401,20.849483,21.292216,21.110874,21.073111,21.071848,...,20.281598,19.920581,19.616548,19.447081,19.167541,19.108408,19.032248,19.116976,18.924928,18.920963
2019-03-10,89,6,19.924768,20.044887,20.077262,20.159969,20.254587,20.194550,20.284663,20.413814,...,20.539652,20.204600,19.908497,19.817461,19.490365,19.496418,19.411085,19.433544,19.419972,19.426788
2019-03-11,89,0,29.024269,33.833840,36.353319,44.146153,47.689857,48.602982,47.334915,46.795633,...,40.259998,37.713417,34.697437,32.557381,31.539096,30.958461,30.819581,30.918707,30.827001,30.487459


### Activity recalculation
Only one step left to get the final dataset: calculate the day's activity. Sundays are inactive by definition, and the rest is divided via the previously calculated threshold

In [13]:
df.insert(2, 'active', True)
for day in df.index:
    mean = df.loc[day, '0':].mean() # Calculate daily consumptions mean
    
    if df.loc[day, 'weekday'] == 6:
        df.loc[day, 'active'] = False
    elif mean > threshold:
        df.loc[day, 'active'] = True
    else:
        df.loc[day, 'active'] = False

df

,building_id,weekday,active,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2012-02-24,89,4,True,33.084816,37.722791,39.619311,46.575444,48.632746,48.472832,47.539280,...,39.977219,37.534106,34.705409,33.102889,32.406356,31.866627,31.853466,31.610200,31.679531,30.957044
2012-02-25,89,5,False,19.842774,19.942503,20.041401,20.849483,21.292216,21.110874,21.073111,...,20.281598,19.920581,19.616548,19.447081,19.167541,19.108408,19.032248,19.116976,18.924928,18.920963
2012-02-26,89,6,False,19.924768,20.044887,20.077262,20.159969,20.254587,20.194550,20.284663,...,20.539652,20.204600,19.908497,19.817461,19.490365,19.496418,19.411085,19.433544,19.419972,19.426788
2012-02-27,89,0,True,29.024269,33.833840,36.353319,44.146153,47.689857,48.602982,47.334915,...,40.259998,37.713417,34.697437,32.557381,31.539096,30.958461,30.819581,30.918707,30.827001,30.487459
2012-02-28,89,1,True,31.603164,36.111243,38.318624,46.232699,50.135373,49.145355,47.396200,...,39.939749,37.473955,34.796319,32.533838,31.695102,31.112563,31.040633,30.835235,31.097579,30.481038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-09,89,5,False,19.842774,19.942503,20.041401,20.849483,21.292216,21.110874,21.073111,...,20.281598,19.920581,19.616548,19.447081,19.167541,19.108408,19.032248,19.116976,18.924928,18.920963
2019-03-10,89,6,False,19.924768,20.044887,20.077262,20.159969,20.254587,20.194550,20.284663,...,20.539652,20.204600,19.908497,19.817461,19.490365,19.496418,19.411085,19.433544,19.419972,19.426788
2019-03-11,89,0,True,29.024269,33.833840,36.353319,44.146153,47.689857,48.602982,47.334915,...,40.259998,37.713417,34.697437,32.557381,31.539096,30.958461,30.819581,30.918707,30.827001,30.487459
